# Synthetic Model 

Notebook to generate a set of models.

#### Import libraries

In [1]:
from IPython.display import Markdown as md
from IPython.display import display as dp
import string as st
import sys
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import datetime

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
from fatiando.utils import ang2vec, vec2ang
from fatiando.mesher import Sphere, Prism, PolygonalPrism
from fatiando.gravmag import sphere, prism, polyprism
from mpl_toolkits.mplot3d import Axes3D

/home/andrelreis/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


In [2]:
notebook_name = 'synthetic_model.ipynb'

#### Importing auxiliary functions

In [3]:
dir_modules = '../../../mypackage'
sys.path.append(dir_modules)

In [4]:
import auxiliary_functions as func

#### Loading 2D grid properties

In [5]:
with open('data/regular_grid.pickle') as f:
        regular = pickle.load(f)

In [6]:
with open('data/airborne_survey.pickle') as f:
        airborne = pickle.load(f)

#### List of saved files

In [7]:
saved_files = []

## Observation area

In [8]:
print 'Area limits: \n x_max = %.1f m \n x_min = %.1f m \n y_max = %.1f m \n y_min = %.1f m' % (regular['area'][1], regular['area'][0],regular['area'][3],regular['area'][2])

Area limits: 
 x_max = 6000.0 m 
 x_min = -6000.0 m 
 y_max = 6000.0 m 
 y_min = -6000.0 m


## Regional Field

In [9]:
inc_gf , dec_gf = (40.,45.) 

## Create a dipping body

In [10]:
model_dip = dict()

### Main field

In [11]:
model_dip['main_field'] = (inc_gf,dec_gf)

## Dipping body vertices

In [12]:
model_dip['verts_x_max'] = 1500.
model_dip['verts_x_min'] = -1500.
model_dip['y_max'] = 2000.
model_dip['y_min'] = -2000
model_dip['z_min'] = 500.
model_dip['z_max'] = 500.
model_dip['z_thick'] = 200.
model_dip['N_prism'] = 41

In [13]:
model_dip['verts_x'] = np.linspace(model_dip['verts_x_min'],
                                   model_dip['verts_x_max'],
                                   model_dip['N_prism'])

In [14]:
model_dip['x_mins'] = []
model_dip['x_maxs'] = []
for i in range(len(model_dip['verts_x'])-1):
    model_dip['x_mins'].append(model_dip['verts_x'][i])
    model_dip['x_maxs'].append(model_dip['verts_x'][i+1])

In [15]:
model_dip['x_mins'] = np.array(model_dip['x_mins'])
model_dip['x_maxs'] = np.array(model_dip['x_maxs'])

In [16]:
model_dip['z_tops'] = np.linspace(model_dip['z_min'],
                                  model_dip['z_max'], 
                                  model_dip['N_prism']-1)
model_dip['z_bottoms'] = np.linspace(model_dip['z_min'] + model_dip['z_thick'],
                                     model_dip['z_max'] + model_dip['z_thick'], 
                                     model_dip['N_prism']-1)

### dip body w/ the presence of remanence magnetization

In [17]:
model_dip['m_R'] = 7.
model_dip['inc_R'] = 30.
model_dip['dec_R'] = 20.

mag_tot_R = ang2vec(model_dip['m_R'],
                    model_dip['inc_R'],
                    model_dip['dec_R'])

model_dip['magnetization_R'] = mag_tot_R

In [18]:
model_dip['magnetization_sph_coord_R'] = vec2ang(mag_tot_R)

In [19]:
model_dip['model_R'] = []
for i in range(model_dip['N_prism']-1):
    model_dip['model_R'].append(Prism(model_dip['x_mins'][i],model_dip['x_maxs'][i],
                           model_dip['y_min'],model_dip['y_max'],
                           model_dip['z_tops'][i],model_dip['z_bottoms'][i],
                           {'magnetization':mag_tot_R}))    

### Dip body w/ purely induced magnetization

In [20]:
model_dip['m_I'] = 7.
model_dip['inc_I'] = inc_gf
model_dip['dec_I'] = dec_gf

mag_tot_I = ang2vec(model_dip['m_I'],model_dip['inc_I'],model_dip['dec_I'])

model_dip['magnetization_I'] = mag_tot_I

In [21]:
model_dip['magnetization_sph_coord_I'] = vec2ang(mag_tot_I)

In [22]:
model_dip['model_I'] = []
for i in range(model_dip['N_prism']-1):
    model_dip['model_I'].append(Prism(model_dip['x_mins'][i],model_dip['x_maxs'][i],
                           model_dip['y_min'],model_dip['y_max'],
                           model_dip['z_tops'][i],model_dip['z_bottoms'][i],
                           {'magnetization':mag_tot_I}))    

### Dip body magnetized at the Pole 

In [23]:
model_dip['main_field_P'] = (90.,0.)

In [24]:
model_dip['m_P'] = 7.
model_dip['inc_P'] = 90.
model_dip['dec_P'] = 0.

mag_tot_P = ang2vec(model_dip['m_P'],model_dip['inc_P'],model_dip['dec_P'])

model_dip['magnetization_P'] = mag_tot_P

In [25]:
model_dip['magnetization_sph_coord_P'] = vec2ang(mag_tot_P)

In [26]:
model_dip['model_P'] = []
for i in range(model_dip['N_prism']-1):
    model_dip['model_P'].append(Prism(model_dip['x_mins'][i],model_dip['x_maxs'][i],
                           model_dip['y_min'],model_dip['y_max'],
                           model_dip['z_tops'][i],model_dip['z_bottoms'][i],
                           {'magnetization':mag_tot_P}))    

#### Generating .pickle file

In [27]:
now = datetime.datetime.utcnow().strftime('%d %B %Y %H:%M:%S UTC')
model_dip['metadata'] = 'Generated by {name} on {date}'.format(date=now, name=notebook_name)

In [28]:
file_name = 'data/model_dip.pickle'
with open(file_name, 'w') as f:
    pickle.dump(model_dip, f)
    
saved_files.append(file_name)

#### Saved files

In [29]:
with open('reports/report_%s.md' % notebook_name[:st.index(notebook_name, '.')], 'w') as q:
    q.write('# Saved files \n')
    now = datetime.datetime.utcnow().strftime('%d %B %Y %H:%M:%S UTC')
    header = 'Generated by {name} on {date}'.format(date=now, name=notebook_name)
    q.write('\n\n'+header+'\n\n')
    for i, sf in enumerate(saved_files):
        print '%d  %s' % (i+1,sf)
        q.write('*  `%s` \n' % (sf))

1  data/model_dip.pickle
